## Kaixuan Wang
## Lab #2

In [0]:
print ("Hello World")

In [0]:
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt

In [0]:
os.chdir('/Users/jessiewang/Desktop/QMSS/5015 Data Analysis/Lab 1') # change working directory
d = pd.read_csv("GSS.2006.csv")
print(d.head())

## 1. Recode 2 variable into new categories.  They can both be continuous-ish or both be nominal-ish, or one of each.  Tell me what you did and explain the variable(s).


## Create a number of categories for religious attendance: low, medium, high ##

In [0]:
## 3 options of low, medium, high ##

conditions = [
    (d['hrsrelax'] < 9) ,
    (d['hrsrelax'] > 8) & (d['hrsrelax'] <17) ,
    (d['hrsrelax'] > 16)]
choices = [1, 2, 3]
d['cut'] = np.select(conditions, choices, default=np.nan)
d.cut.describe()

In [0]:
# Look at the results

d.cut.describe()

In [0]:
## How many of each category are there?

d.cut.value_counts()

## Check the recoding:  It is "1" for categories less than 3, it is "2" for categories from 3 to 5, and it is "3" for categories greater than 5.

In [0]:
res = pd.crosstab(d.attend, d.cut)
res.astype('float').div(res.sum(axis=0), axis=1)

In [0]:
#Simply another way to do the same thing as above:  A nice function someone wrote to do the same thing:

def binning(col, cut_points, labels=None):
  #Define min and max values:
  minval = col.min()
  maxval = col.max()

  #create list by adding min and max to cut_points
  break_points = [minval] + cut_points + [maxval]

  #if no labels provided, use default labels 0 ... (n-1)
  if not labels:
    labels = range(len(cut_points)+1)

  #Binning using cut function of pandas
  colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
  return colBin

#Binning attend:

cut_points = [2,5]
labels = ["low","medium","high"]
d["attend_cut"] = binning(d["attend"], cut_points, labels)
##print pd.value_counts(d["attend_cut"], sort=False)##


In [0]:
## See it works the same way...

summary = d.attend_cut.describe()
summary = summary.transpose()
summary

In [0]:
## See it works the same way...

d.attend_cut.value_counts()

In [0]:
## See it works the same way...

res = pd.crosstab(d.attend, d.attend_cut)
res.astype('float').div(res.sum(axis=0), axis=1)

## Below is a binary recode, where we make it 1 if "How often do you attend religious services?" is everything as frequent as 4 or more, which is About once a month or more, 0 otherwise

In [0]:
##Simple binary cut##

conditions = [
    (d['attend'] < 5) ,
    (d['attend'] >4 )]
choices = [0,1]
d['high'] = np.select(conditions, choices, default=np.nan)

## Check the recoding:  It is 0 for all categories less than 5 and 1 for everything greater than or equal to 5

In [0]:
res = pd.crosstab(d.attend, d.high)
res.astype('float').div(res.sum(axis=0), axis=1)

## Let's look at another variable - ## this is a question about "To what extent do you agree or disagree with the following statements? I see myself as someone who... c. Does a thorough job", with 1 being strongly agree and 5 being strongly disagree.

In [0]:
## Table 2 ##

my_tab = pd.crosstab(index=d["big5c1"],  # Make a crosstab
                              columns="count")     
def compute_percentage(x):
      pct = float(x/my_tab['count'].sum()) * 100
      return round(pct, 2)

my_tab['percentage'] = my_tab.apply(compute_percentage, axis=1)

my_tab

In [0]:
d.hist(column='big5c1')

##  People who say they strongly agree are coded 1 for thorough, otherwise 0.

In [0]:
conditions = [
    (d['big5c1'] < 2) ,
    (d['big5c1'] >=2 )]
choices = [1,0]
d['thorough'] = np.select(conditions, choices, default=np.nan)

In [0]:
my_tab_thor = pd.crosstab(index=d["thorough"],  # Make a crosstab
                              columns="count")     

def compute_percentage(x):
      pct = float(x/my_tab['count'].sum()) * 100
      return round(pct, 2)

my_tab_thor['percentage'] = my_tab_thor.apply(compute_percentage, axis=1)

my_tab_thor

## 2. Use one (or both) of your recoded variables to do a cross-tabulation.  Explain your results.

## The below results indicate that being more thorough is not associated with going to religious services more (low-religious people are thorough at 48.5%, while high religious people are thorough at 49.7% ~ essentially the same), which is contrary to what some psychological theories would have predicted

In [0]:
res = pd.crosstab(d.thorough, d.high)
res.astype('float').div(res.sum(axis=0), axis=1)

## BTW: You can use pandas "crosstab" function to get column percentages now

In [0]:
res.p = pd.crosstab(d['thorough'], d['high'], normalize='columns')
res.p

## 3. Run a linear regression with 1 independent and 1 dependent variable; make all of the recodes necessary to make the model as easy to interpret as possible; and explain your results.

## Asking the question: How do you think one's hours should be related to one's spouse's hours?

In [0]:
lm = smf.ols(formula = 'hrs1~sphrs1', data = d).fit()
print (lm.summary()) 

## Conclusion: Knowing your spouse's hours at work do not help predicting your hours at work.  A coefficient of 0.04662 indicates no obvious relationship between my hours and my spouse's hours.

## 4. Plot two variables, either as a scatter plot or boxplot; add in trend/regression lines; and explain your results.

## Here is plotting of sphrs1 against hrs1 using seaborn:

In [0]:
import seaborn as sns
sns.regplot(d['hrs1'],d['sphrs1'])